# Segmenting and Clustering Neighborhoods in Toronto

## Part 1. Retreiving the data online

In [1]:
# !pip install beautifulsoup4
# !pip install requests

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

### 1. Get data from Wikipedia and convert to bs4 object

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(url, 'html.parser')

### 2. Scrape data from table to lists

In [5]:
# store the postal code, borough and neighborhood in lists

postalcode = []
borough = []
neighborhood = []

for row in soup.find('table').find('tbody').find_all('tr'):
    for col in row.find_all('td'):
        if col.span.text == 'Not assigned': 
            pass
        else:
            postalcode.append(col.find('b').text)
            cell = col.span.text.split('(')
            borough.append(cell[0])
            neighborhood.append(cell[1].replace(")","").strip(" ").replace('/',','))
        
# checking if all lists have the same dimension
print(len(postalcode),len(borough),len(neighborhood))


103 103 103


### 3. Convert into dataframe

In [6]:
# converting data into dataframe

lst = [postalcode, borough, neighborhood]

df = DataFrame(lst).transpose()
df.columns = ['Postal Code', 'Borough', 'Neighborhood']
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Checking if there are any 'Not assigned' under the 'Neighborhood' column

In [7]:
df.loc[df['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


In [8]:
# shape of dataframe
df.shape

(103, 3)

## Part 2. Getting geolocation data

In [9]:
#!pip install geocoder

Attempt to use geocoder API:

In [10]:
import geocoder

In [11]:
# # initialize your variable to None
# lat_lng_coords = None

# # test out one postalcode
# postal_code = 'M7A'

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     print(g.latlng)
#     lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

Realized that geocoder isn't working; import csv file instead

In [12]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
newdf = pd.merge(df, geodata, how='inner', on='Postal Code')
newdf

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
